# NAVM.jl 测试

✨Powered by [**IpynbCompile.jl**](https://github.com/ARCJ137442/IpynbCompile.jl)

## 模块导入

模块清单：

- JuNarsese
- NAVM
- Test

In [1]:
include("#import.jl")

JuNarsese v2.6.3


┌ Info: NAVM导入成功！
│   names(NAVM) =
│    33-element Vector{Symbol}:
│     :Backend
│     :BackendModule
│     :CMD_CYC
│     :CMD_DEL
│     :CMD_HLP
│     ⋮
│     :target_type
│     :transform
│     :try_form_cmd
│     :try_transform
└     :tryparse_cmd
Precompiling NAVM_Implements
        Info Given NAVM_Implements was explicitly requested, output will be shown live 
JuNarsese v2.6.3
  ✓ NAVM_Implements
  1 dependency successfully precompiled in 4 seconds. 20 already precompiled.
  1 dependency had output during precompilation:
┌ NAVM_Implements
│  [Output was shown above]
└  


"all"

## 测试 / 前端

In [2]:
fea = FE_TextParser(StringParser_ascii)
fes = FE_TextParser(SExprParser)
@info "✅测试用前端 构建完成" fea fes

┌ Info: ✅测试用前端 构建完成
│   fea = FE_TextParser(StringParser_ascii)
└   fes = FE_TextParser(SExprParser)


In [3]:
#= %only-compiled # ! 前端测试组 上下文 开始
@testset "前端" begin
%only-compiled =#

In [4]:
let cmds = transform(fea, "<A --> B>.")
    @test cmds |> length == 1
    @test cmds[1] isa NAVM.CMD_NSE
end

let cmds = fea("/VOL 10")
    @test cmds |> length == 1
    @test cmds[1] isa NAVM.CMD_VOL
    @test cmds[1].volume == 10
end

let cmds = fes([
    "(Inheritance (Word A) (Word B))"
    "12"
])
    @test cmds[1] isa NAVM.CMD_NSE
    @test cmds[1].narsese isa JuNarsese.AbstractTerm
    @test cmds[1].narsese isa JuNarsese.Inheritance
    @test cmds[1].narsese[1] isa JuNarsese.Word
    @test cmds[1].narsese[2] isa JuNarsese.Word
    @test cmds[1].narsese[1] |> JuNarsese.nameof == :A
    @test cmds[1].narsese[2] |> JuNarsese.nameof == :B

    @test cmds[2] isa NAVM.CMD_CYC
    @test cmds[2].cycles == 12
end

Test Passed

In [5]:
#= %only-compiled # ! 前端测试组 上下文 结束
end # begin @test
%only-compiled =#

## 测试 / 后端

In [6]:
ben = BE_OpenNARS()
beo = BE_ONA()
bep = BE_NARS_Python()
bey = BE_PyNARS()
bej = BE_OpenJunars()
@info "✅测试用后端 构建完成" ben beo bep bey bej

┌ Info: ✅测试用后端 构建完成
│   ben = BE_OpenNARS()
│   beo = BE_ONA()
│   bep = BE_NARS_Python()
│   bey = BE_PyNARS()
└   bej = BE_OpenJunars()


In [7]:
#= %only-compiled # ! 后端测试组 上下文 开始
@testset "后端" begin
%only-compiled =#

In [8]:
# 轮流测试
testset = [
    NAVM.CMD_REM("这是一段不会被翻译的注释")
    NAVM.CMD_NSE(nse"<A --> B>.")
    NAVM.CMD_NSE(nse"<B --> C>.")
    NAVM.CMD_CYC(5)
    # NAVM.CMD_UNK(5, Operator, Int, :a123, r"123", nse"<A --> B>.")
]
for be in [ben, beo, bep, bey, bej]
    @info "开始测试$(be)。。。"
    let result = be.(testset),
        result_combined = be(testset)
        @test length(result_combined) <= 4 # 不会超过四条
        # 测试Narsese的共有部分
        @test occursin("A --> B", result[2][1])
        @test occursin("B --> C", result[3][1])
        if be !== bep # NARS-Python 没有CYC
            @test occursin("5", result[4][1])
        end
        @info "测试成功！"
    end
end

[ Info: 开始测试BE_OpenNARS()。。。
[ Info: 测试成功！
[ Info: 开始测试BE_ONA()。。。
[ Info: 测试成功！
[ Info: 开始测试BE_NARS_Python()。。。
[ Info: 测试成功！
[ Info: 开始测试BE_PyNARS()。。。
[ Info: 测试成功！
[ Info: 开始测试BE_OpenJunars()。。。
[ Info: 测试成功！


In [9]:
#= %only-compiled # ! 后端测试组 上下文 结束
end # begin @test
%only-compiled =#

# 测试 / 前后端协同

In [10]:
#= %only-compiled # ! 前后端协同测试组 上下文 开始
@testset "前后端协同" begin
%only-compiled =#

In [11]:
testset_a = [
    # 多行文本测试
    """
    # 下面输入一段基本的三段论推理
    # 基本逻辑：「A是B」「B是C」⇒「A是C」
    """
    "<A -->B>."
    "<B--> C>."
    "<A--> C>?"
    "50"
]
testset_s = """
/RES memory
/LOA memory *
# 下面输入一段基本的三段论推理
# 基本逻辑：「A是B」「B是C」⇒「A是C」
(SentenceJudgement (Inheritance (Word A) (Word B)) )
(SentenceJudgement (Inheritance (Word B) (Word C)) )
(SentenceQuestion  (Inheritance (Word A) (Word C)) )
/VOL 10
50
/CYC 50
/INF memory
/SAV memory *
"""

let chain_aj = chain(fea, bej)
    result = chain_aj(testset_a)
    @test chain_aj isa Function
    @test occursin("A --> B", result[1])
    @test occursin(".", result[1])
    @test occursin("B --> C", result[2])
    @test occursin(".", result[2])
    @test occursin("A --> C", result[3])
    @test occursin("?", result[3])
    @test occursin("50", result[4])
end

for be in [ben, beo, bep, bey, bej]
    printstyled("字串 |> $(fes) |> $(be) = \n"; color=:light_cyan, bold=true)
    let result = join((be ∘ fes)(testset_s), "\n")
        # OpenNARS、ONA的音量指令
        if be in [ben, beo]
            @test occursin("*volume=10", result)
        end
        # 周期指令
        if be === bej
            # OpenJunars的周期指令
            @test occursin(":c", result)
        elseif be !== bep
            # 非NARS-Python的周期指令
            @test occursin("50", result)
        end
        # Narsese
        @test occursin("A --> B", result)
        @test occursin("B --> C", result)
        @test occursin("A --> C", result)
        @test occursin(".", result)
        @test occursin("?", result)
        # 打印结果
        result |> println
    end
end

字串 |> FE_TextParser(SExprParser) |> BE_OpenNARS() = 


┌ Debug: BE_OpenNARS(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_OpenNARS(): 未知的指令「CMD_LOA("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_OpenNARS(): 未知的指令「CMD_INF("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_OpenNARS(): 未知的指令「CMD_SAV("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
*volume=10
50
50
字串 |> FE_TextParser(SExprParser) |> BE_ONA() = 
*reset
*load
<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
*volume=10
50
50
*save
字串 |> FE_TextParser(SExprParser) |> BE_NARS_Python() = 


┌ Debug: BE_ONA(): 未知的指令「CMD_INF("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


load
(A --> B). %1.0;0.5%
(B --> C). %1.0;0.5%
(A --> C)?
count
save


┌ Debug: BE_NARS_Python(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_NARS_Python(): 未知的指令「CMD_VOL(10)」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


字串 |> FE_TextParser(SExprParser) |> BE_PyNARS() = 
<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
50
50


┌ Debug: BE_PyNARS(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_PyNARS(): 未知的指令「CMD_LOA("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_PyNARS(): 未知的指令「CMD_VOL(10)」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_PyNARS(): 未知的指令「CMD_INF("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_PyNARS(): 未知的指令「CMD_SAV("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


字串 |> FE_TextParser(SExprParser) |> BE_OpenJunars() = 
<A --> B>. %1.0;0.5%
<B --> C>. %1.0;0.5%
<A --> C>?
:c 50
:c 50
:p


┌ Debug: BE_OpenJunars(): 未知的指令「CMD_RES("memory")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_OpenJunars(): 未知的指令「CMD_LOA("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_OpenJunars(): 未知的指令「CMD_VOL(10)」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17
┌ Debug: BE_OpenJunars(): 未知的指令「CMD_SAV("memory", "*")」
└ @ NAVM_Implements h:\A137442\Develop\_NAR\NAVM_Implements\src\backend\common.jl:17


In [12]:
#= %only-compiled # ! 前后端协同测试组 上下文 结束
end # begin @test
%only-compiled =#

## 尝试自编译

In [13]:
# %ignore-cell
using IpynbCompile: compile_notebook
THIS_PATH = "runtests.ipynb"
TARGET_PATH = "runtests.jl"
compile_notebook(THIS_PATH, TARGET_PATH)

┌ Debug: Loading object cache file C:\Users\Administrator\.julia\compiled\v1.10\IpynbCompile\pIVMh_pEIZk_1.dll for IpynbCompile [4eb781bf-a71e-403a-9d46-9d48649f04b2]
└ @ Base loading.jl:1051


4561